- scoring : ('roc_auc') is only used in binary clasification 

In [46]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer, make_column_selector 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score, r2_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [74]:
sat = pd.read_csv('Satellite.csv',sep=";")
sat.head(10)

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil
5,80,94,98,76,80,102,102,79,76,102,...,79,79,107,109,87,79,107,109,87,grey soil
6,76,102,106,83,76,102,106,87,80,98,...,87,79,103,104,83,79,103,104,79,grey soil
7,76,102,106,87,80,98,106,79,76,94,...,83,79,103,104,79,79,95,100,79,grey soil
8,76,89,98,76,76,94,98,76,76,98,...,75,75,91,96,71,79,87,93,71,damp grey soil
9,76,94,98,76,76,98,102,72,76,94,...,71,79,87,93,71,79,87,93,67,damp grey soil


In [38]:
X = sat.drop(columns='classes')
y = sat.classes

In [39]:
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=24, test_size=0.3,stratify=y)

In [32]:
# LDA using accuracy score
lda = LinearDiscriminantAnalysis()

lda.fit(X_train,y_train)
y_pred = lda.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8296219575349559


In [33]:
# using predict_proba
y_pred = lda.predict_proba(X_test)
print(log_loss(y_test,y_pred))

0.5367082791863951


In [34]:
# QDA using accuracy score
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train,y_train)
y_pred = qda.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8508544795442776


In [35]:
# using predict_proba
y_pred = qda.predict_proba(X_test)
print(log_loss(y_test,y_pred))

0.9634636510805074


In [62]:

knn = KNeighborsClassifier(n_neighbors=3)
pipe = Pipeline([('LDA',lda),('KNN',knn)])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8663904712584153


In [83]:
knn = KNeighborsClassifier()
pipe = Pipeline([('LDA',lda),('KNN',knn)])
kfold = StratifiedKFold(n_splits=5, random_state=24, shuffle=True)
params = {'KNN__n_neighbors' : np.arange(1,9)}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold)   
# scoring : ('roc_auc') is only used in binary clasification 
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis()),
                                       ('KNN', KNeighborsClassifier())]),
             param_grid={'KNN__n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8])})

In [86]:
print(gcv.best_params_)
print(gcv.best_score_)

{'KNN__n_neighbors': np.int64(7)}
0.8822066822066823
